# Functions

**The core idea:** A function is a named, reusable block of logic. Everything we've covered today — lists, dicts, sets, try/except — lives inside functions in real code. This is where it all comes together.

---

**The anatomy:**


In [1]:
def println()->None:
    print('-'*50)

In [4]:
def greet(name, greeting="Hello"):   # greeting has a default
    result = f"{greeting}, {name}!"
    return result                     # sends value back to caller

print(greet("Alice"))           # "Hello, Alice!"
println()
print(greet("Alice", "Hey") )   # "Hey, Alice!"

Hello, Alice!
--------------------------------------------------
Hey, Alice!


`def` declares it. Parameters are inputs. `return` sends a value back. Without `return` the function returns `None` silently — a common bug.

---

**Multiple return values — tuple unpacking:**

In [9]:
from typing import Tuple
def min_max(nums)->Tuple:
    return min(nums), max(nums)   # returns a tuple

low, high = min_max([3, 1, 4, 1, 5])
print(low, high)    # 1 5

1 5


**`*args` and `**kwargs` — flexible arguments:**
# args is a tuple of all positional args
# kwargs is a dict of keyword args

In [10]:
def add_all(*args):          # args is a tuple of all positional args
    return sum(args)

add_all(1, 2, 3, 4)         # 10

10

In [11]:
def display(**kwargs):       # kwargs is a dict of keyword args
    for k, v in kwargs.items():
        print(f"Name is {k}: Age is {v}")

display(name="Alice", age=30)

Name is name: Age is Alice
Name is age: Age is 30


**Scope — where variables live:**

In [12]:
x = 10          # global scope

def change():
    x = 99      # local scope — does NOT affect global x
    print(x)    # 99

change()
print(x)        # still 10

99
10


# Variables inside a function are local — they don't leak out. If you need to modify a global variable inside a function use `global x` but avoid this — it's a design smell.

In [15]:
# bad practice avoid if you can 
x = 10          # global scope

def change():
    global x
    x = 99      # local scope — does NOT affect global x
    print(x)    # 99

change()
print(x)        # still 10

99
99


**Lambda — anonymous one-liner functions:**

In [16]:
square = lambda x: x ** 2
square(5)    # 25

25

In [17]:
nums = [3, 1, 4, 1, 5]
nums.sort(key=lambda x: x ** 2)  # sort by squared value
print(nums)

[1, 1, 3, 4, 5]


In [18]:
# if/else in one line
classify = lambda x: "positive" if x > 0 else "negative" if x < 0 else "zero"
classify(-5)    # 'negative'

'negative'

In [20]:
# Most common use — as a sort key
words = ["banana", "fig", "apple", "kiwi"]
words.sort(key=lambda w: len(w))    # sort by length
print(words)
# ["fig", "kiwi", "apple", "banana"]

['fig', 'kiwi', 'apple', 'banana']


In [23]:
pairs = [(1, 3), (2, 1), (3, 2)]
pairs.sort(key=lambda p: p[1])      # sort by second element
print(pairs)
println()
pairs.sort(key=lambda p: p[0]) 
print(pairs)
println()
# [(2,1), (3,2), (1,3)]

[(2, 1), (3, 2), (1, 3)]
--------------------------------------------------
[(1, 3), (2, 1), (3, 2)]
--------------------------------------------------


---

**Functions as first-class objects** — you can pass them around:

In [25]:
def apply(func, value):
    return func(value)

print(apply(str.upper, "hello") )   # "HELLO"
println()
print(apply(len, "hello")   )       # 5

HELLO
--------------------------------------------------
5


**LeetCode patterns — functions you'll write constantly:**

**Two pointer inside a function:**

In [26]:
def is_palindrome(s):
    left, right = 0, len(s) - 1
    while left < right:
        if s[left] != s[right]:
            return False
        left += 1
        right -= 1
    return True

**Frequency check:**

In [32]:
from collections import Counter  # ✅ lowercase 'c', plural 's'

def is_anagram(s, t):
    if len(s) != len(t):
        return False
    return Counter(s) == Counter(t)

print(is_anagram("ate", "eat"))  # True
print(is_anagram("ate", "eats"))  # False

True
False


**Helper functions — break complex problems into steps:**

Perfect summary. Let me just make it concrete:

```python
def process(data):
    
    def clean(x):          # only visible inside process()
        return x.strip().lower()
    
    return [clean(item) for item in data]
```

`clean()` doesn't exist outside of `process()` — try calling it from outside and you get a `NameError`.

The encapsulation point you made is key — because `clean` can't reach outside its scope to grab globals, **everything it needs must come in through params, everything it produces goes out through return**. This forces clean, predictable, testable code with no hidden side effects.

The practical reasons you'd use an inner function over a lambda:
- It needs more than one line
- It needs a name for readability/self-documentation  
- You want to reuse it in multiple places *within* the outer function
- You want to keep it private — not expose it as part of the module's public interface

In [34]:
def process(nums):
    def is_valid(n):        # inner function — only visible inside process
        return n > 0

    return [n for n in nums if is_valid(n)]

**Recursion — a function that calls itself:**

In [35]:
def factorial(n):
    if n == 0:              # base case — always need this
        return 1
    return n * factorial(n - 1)   # recursive case

factorial(5)    # 120

120

In [38]:
from collections import defaultdict
fibs = defaultdict(int)
cache = {}
def fib(n):
    # TODO: Implement cached Fibonacci
    if n in cache: return cache[n]
    if n <= 1: return n
    res = fib(n-1) + fib(n-2)
    cache[n] = res
    return res
    
for i in range( 11):
    fibs[i] = fib(i)
print(fibs.keys())
print(fibs.values())

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
dict_values([0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55])


**use `functools.lru_cache` (memoization decorator). It handles the cache internally, no global needed:**
`@lru_cache` wraps the function and maintains the cache **inside the decorator itself** — completely invisible to you, no global, no manual `cache[n] = res`. Each recursive call still hits the same cache because the decorator lives on the function object, not in any scope.

`maxsize=None` means unlimited cache size. You can also write it as `@cache` in Python 3.9+:

This is the Pythonic standard for memoized recursion — clean, no side effects, no globals.

In [43]:
from functools import lru_cache
from collections import defaultdict

@lru_cache(maxsize=None)
def fib(n):
    if n <= 1: return n
    return fib(n-1) + fib(n-2)



print(fib(6))

8


In [44]:
def mystery(a, b=10):
    return a * b

print(mystery(5))
print(mystery(5, 2))

def swap(lst, i, j):
    lst[i], lst[j] = lst[j], lst[i]

nums = [1, 2, 3]
swap(nums, 0, 2)
print(nums)

50
10
[3, 2, 1]
